In [1]:
import warnings
warnings.filterwarnings('ignore')
import locale
locale.getpreferredencoding = lambda x=False: "UTF-8"
! pip install -q transformers datasets evaluate
from transformers import AutoModelForCausalLM, BloomTokenizerFast, set_seed
# device = 'cuda'
import torch
import gc
device = 'cuda'
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00


In [2]:
def get_model(model_name):
  model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
  return model

def tokenize(text):
   return tokenizer.encode(text, return_tensors="pt").to( device)

tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloom")


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [3]:
import numpy as np

def get_class_scores(target_id_mapping, scores):
  target_scores = {target: 0 for target in target_id_mapping.keys()}
  for score in scores:
    for target in target_scores.keys():
       target_scores[target] += score[0][target_id_mapping[target]]
  return target_scores

def get_classification(model, example):
    text = example.get('text')
    target_ids = [tokenize(str(i))[0][0] for i, _ in enumerate(example.get('options'))]
    target_id_mapping = {i: target_ids[i] for i, target in enumerate(example.get('options'))}
    inputs = tokenizer.encode(text, return_tensors="pt").to(device)
    outputs = model.generate(inputs, return_dict_in_generate=True, min_new_tokens=1, max_new_tokens=1, output_scores=True)
    generated_tokens = outputs.sequences[0][inputs.shape[1]:]
    response = tokenizer.decode(generated_tokens)
    target_scores = get_class_scores(target_id_mapping, outputs.scores)
    example['prediction'] = max(target_scores, key= lambda x: target_scores[x])
    example['response'] = response
    return example

def classify_tweets(model, dataset):
    return dataset.map(lambda example: get_classification(model, example))

In [4]:
models = ['bigscience/bloomz-1b7', 'bigscience/bloom-1b7', 'alonzogarbanzo/Bloom-1b7-winograd-wsc-IT-baseline', "alonzogarbanzo/Bloom-1b7-ropes-Cont-IT-Step2", "alonzogarbanzo/Bloom-1b7-glue-mrpc-Cont-IT-Step3", "alonzogarbanzo/Bloom-1b7-dialogsum-Cont-IT-Step4", "alonzogarbanzo/Bloom-1b7-creative-writing-Cont-IT-Step5"]

In [5]:
import evaluate
def calc_f1_score(dataset):
    predictions = dataset['prediction']
    references = dataset['label']
    f1 = evaluate.load("f1")
    return f1.compute(predictions=predictions, references=references, average='macro')

def calc_accuracy(dataset):
    predictions = dataset['prediction']
    references = dataset['label']
    accuracy = evaluate.load("accuracy")
    return accuracy.compute(predictions=predictions, references=references)

def calc_precision(dataset):
    predictions = dataset['prediction']
    references = dataset['label']
    precision = evaluate.load("precision")
    return precision.compute(predictions=predictions, references=references, average='macro')

def calc_recall(dataset):
    predictions = dataset['prediction']
    references = dataset['label']
    recall = evaluate.load("recall")
    return recall.compute(predictions=predictions, references=references , average='macro' )


In [6]:
import json
def evaluate_models(models, dataset_name, dataset):
  evaluation_data = {model_name: {} for model_name in models}
  evaluation_metrics = {model_name: {} for model_name in models}
  for model_name in models:
    model = get_model(model_name)
    prediction_dataset = classify_tweets(model, dataset)
    evaluation_data[model_name]['f1'] = f1 = calc_f1_score(prediction_dataset)
    evaluation_data[model_name]['accuracy'] = accuracy = calc_accuracy(prediction_dataset)
    evaluation_data[model_name]['precision'] = precision = calc_precision(prediction_dataset)
    evaluation_data[model_name]['recall'] = recall = calc_recall(prediction_dataset)
    prediction_dataset.to_csv(f'./winograd/{model_name[15:]}_{dataset_name}.csv')
    evaluation_metrics[model_name]['f1'] = f1.get('f1')
    evaluation_metrics[model_name]['accuracy'] = accuracy.get('accuracy')
    evaluation_metrics[model_name]['precision'] = precision.get('precision')
    evaluation_metrics[model_name]['recall'] = recall.get('recall')
    del model
    gc.collect()
    torch.cuda.empty_cache()
    print(f'{model_name} {dataset_name} evaluation complete.\n {evaluation_metrics[model_name]}')

    with open(f'./winograd/evaluation_metrics_{dataset_name}.json', 'w') as f:
      json.dump(evaluation_metrics, f, indent=4)
  return evaluation_data

In [7]:
def transform_dataset(dataset, transform_function):
  transformed_data = dataset.map(lambda example: transform_function(example))
  return transformed_data

In [8]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def display_cm(dataset, model_name):
  predictions = dataset['prediction']
  references = dataset['label']
  class_labels = [0, 1, 2]
  cm = confusion_matrix(references, predictions, labels=class_labels)
  plt.figure(figsize=(8, 6))
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
  disp.plot()
  plt.title(f'{model_name} Confusion Matrix')
  plt.show()


In [9]:
import random
def transform(example):
    prompts = [
      "Determine which option the pronoun refers to in this text: ",
      "Given the text, identify the referent of the pronoun among these options: ",
      "Read the text and decide which option is referred to by the pronoun: ",
      "In the text below, to whom or what does the pronoun refer? Choose from the options: ",
    ]
    prompt = random.choice(prompts)
    option_text = ""
    for i, option in enumerate(example['options']):
        option_text += f'{i}: {option}\n'
    input_text = f"{prompt}Text: '{example.get('text')}' Pronoun: '{example.get('pronoun')}', Quote: '{example.get('quote')}'. {option_text}. Response with the correct option number only. Option: "
    example["text"] = f'''Given the following text:
    {example.get('stext')}
    What entity does "{example.get('pronoun')}" refer to from these options:
    {option_text}

    Give your answer as the option number
    Answer: '''
    return example

In [10]:
dataset_name = "winograd_wsc"

In [11]:
dataset = load_dataset("adambjorn/UnrelatedForgettingOverhead", data_dir=dataset_name, split="test")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [12]:
dataset = transform_dataset(dataset, transform)
data = evaluate_models(models, dataset_name, dataset)

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

alonzogarbanzo/Bloom-1b7-creative-writing-Cont-IT-Step5 winograd_wsc evaluation complete.
 {'f1': 0.358974358974359, 'accuracy': 0.5066666666666667, 'precision': 0.33585858585858586, 'recall': 0.42777777777777776}


In [13]:
import os
import zipfile

def zip_directory(folder_path, output_filename):
    # Create a zipfile object in write mode
    with zipfile.ZipFile(output_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # The os.walk function allows us to walk through the directory tree
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create the file path by joining the directory and file name
                file_path = os.path.join(root, file)
                # Create the archive name, which is the path within the zip file
                # This is the path relative to the folder we are zipping
                arcname = os.path.relpath(file_path, os.path.dirname(folder_path))
                # Write the file to the zipfile
                zipf.write(file_path, arcname)

# Example usage
folder_path = './winograd'  # Replace with the path to your directory
output_filename = 'winograd.zip'  # Replace with your desired output zip file name
zip_directory(folder_path, output_filename)
